# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

//anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [ ]:
data = pd.read_csv('C:/Users/vkhur/Desktop/Учеба/Рекомендательные системы/Lesson2/webinar_2-20220325T134131Z-001/webinar_2/retail_train.csv')
item_features = pd.read_csv('C:/Users/vkhur/Desktop/Учеба/Рекомендательные системы/Lesson2/webinar_2-20220325T134131Z-001/webinar_2/product.csv')
user_features = pd.read_csv('C:/Users/vkhur/Desktop/Учеба/Рекомендательные системы/Lesson2/webinar_2-20220325T134131Z-001/webinar_2/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [ ]:
recommender = MainRecommender(data_train_lvl_1)

In [ ]:
recommender.get_als_recommendations(2375, N=5)

[899624, 1106523, 1044078, 871756, 844179]

In [ ]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [ ]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 1044078, 1044078, 1078652, 1018809]

In [ ]:
recommender.get_similar_users_recommendation(2375, N=5)

[1101502, 979674, 10457044, 974265, 959455]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [ ]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [ ]:
# your_code

N = 200
hot_users = data_train_lvl_1['user_id'].unique().tolist()
top_popular = recommender.overall_top_purchases[:N]

In [ ]:
%%time

result_lvl_1['als'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=N) if x in hot_users else top_popular)

In [ ]:
%%time

result_lvl_1['self'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=N) if x in hot_users else top_popular)

In [ ]:
%%time

result_lvl_1['similar_items'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=N) if x in hot_users else top_popular)

In [ ]:
result_lvl_1['combined'] = result_lvl_1['user_id'].apply(lambda x: \
    result_lvl_1.loc[result_lvl_1.user_id == x]['als'].tolist()[0][0:66] + \
    result_lvl_1.loc[result_lvl_1.user_id == x]['self'].tolist()[0][0:67] + \
    result_lvl_1.loc[result_lvl_1.user_id == x]['similar_items'].tolist()[0][0:67])

result_lvl_1.head()

In [ ]:
def average_recall_at_k(recommended_matrix, bought_matrix, k=5):
    
    rows_count = bought_matrix.shape[0]
    recall_by_row = [recall_at_k(recommended_matrix[i], bought_matrix[i], k) for i in range(rows_count)]
    recall_mean = np.mean(recall_by_row)

    return recall_mean

In [ ]:
columns = result_lvl_1.columns.drop(['user_id', 'actual'])

for column in columns:
    recall_mean = average_recall_at_k(result_lvl_1[column], result_lvl_1.actual, k=200)
    print('{:35} {:.4f} %'.format(column, recall_mean * 100))

Лучший результат дает сочетание рекомендации собственных покупок и топ популярных покупок.

In [ ]:
# Перебираемые значения K.
k_list = [20, 50, 100, 200, 300, 400, 500]

# Список для сохранения результатов.
recall_list = [] # 

# Список пользователей.
users_list = result_lvl_1['user_id'].values

for k in k_list:
    own_recs = []
    for user in users_list: 
        own_recs.append(recommender.get_own_recommendations(user, N=k) if x in hot_users else top_popular)
    recall_list.append(average_recall_at_k(own_recs, result_lvl_1.actual, k=k))
    

plt.figure(figsize=(10, 6))

plt.plot(k_list, recall_list, label="Own recommendation")
plt.xlabel('k-items')
plt.ylabel('recall@k')
plt.legend()
plt.grid()
plt.show()

Значение метрики перестает расти, если K>200. Скорее всего это связано с тем что в среднем из валидационного датасета пользователи покупают 12 товаров. При K>200 все купленные товары попадают в число рекомендованных.

In [ ]:
np.unique(data_val_lvl_1.item_id.values).size / np.unique(data_val_lvl_1.user_id.values).size

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [ ]:
# your_code
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]


In [ ]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']
model_rec = recommender.get_model_recommendation(N=200)

In [ ]:
users_lvl_2 = users_lvl_2.merge(model_rec,
                                on='user_id',
                                how='inner')

users_lvl_2.columns = ['user_id', 'candidates']
users_lvl_2.head()

In [ ]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1)\
    .stack().reset_index(level=1, drop=True)

s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.head(4)

In [ ]:
data_train_lvl_2.head(2)

In [ ]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id', 'quantity', 'sales_value', 'store_id', 'week_no']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

targets_lvl_2.head()

In [ ]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

targets_lvl_2.head(2)

In [ ]:
targets_lvl_2['quantity'].fillna(targets_lvl_2['quantity'].median(),
                                 inplace=True)
targets_lvl_2['sales_value'].fillna(targets_lvl_2['sales_value'].mean(),
                                    inplace=True)

In [ ]:
def calc_mode_func_series(x, mode_store):
    x = pd.Series.mode(x)
    if type(x) != float:
        if len(x) >= 1:
            x = x[0]
        else:
            x = mode_store
    return x

mode_store = pd.Series.mode(targets_lvl_2['store_id']).values[0]

df = \
    targets_lvl_2.groupby(by='user_id')['store_id']\
        .agg(lambda x: calc_mode_func_series(x, mode_store)).reset_index()

df.rename(columns={'store_id': 'mode_store_user'},
          inplace=True)

targets_lvl_2 = targets_lvl_2.merge(df, 
                                    on='user_id',
                                    how='inner')

targets_lvl_2.head(2)

In [ ]:
df = pd.pivot_table(targets_lvl_2,
                    index='item_id', columns='week_no',
                    values='quantity',
                    aggfunc='count',
                    fill_value=0
                    )

df = df.agg('median', axis='columns').reset_index()
df.columns = ['item_id', 'quantatity_of_item_per_week']

targets_lvl_2 = targets_lvl_2.merge(df,
                                    on='item_id',
                                    how='inner')

targets_lvl_2.head(2)

In [ ]:
df = pd.pivot_table(targets_lvl_2,
                    index='department', columns='week_no',
                    values='quantity',
                    aggfunc='count',
                    fill_value=0
                    )

df = df.agg('median', axis='columns').reset_index()
df.columns = ['department', 'quantatity_of_item_in_category_per_week']

targets_lvl_2 = targets_lvl_2.merge(df,
                                    on='department',
                                    how='inner')
targets_lvl_2.head(2)

In [ ]:
df = pd.pivot_table(targets_lvl_2,
                    index='user_id', columns='department',
                    values='quantity',
                    aggfunc='count',
                    fill_value=0
                    )

df = df.idxmax(axis=1).reset_index()
df.columns = ['user_id', 'top_department']

targets_lvl_2 = targets_lvl_2.merge(df,
                                    on='user_id',
                                    how='inner'
                                    )

In [ ]:
df = pd.pivot_table(targets_lvl_2,
                    index='user_id', columns='brand',
                    values='quantity',
                    aggfunc='count',
                    fill_value=0
                    )

df = df.idxmax(axis=1).reset_index()
df.columns = ['user_id', 'top_brand']

targets_lvl_2 = targets_lvl_2.merge(df,
                                    on='user_id',
                                    how='inner'
                                    )

df = pd.pivot_table(targets_lvl_2,
                    index='user_id', columns='department',
                    values='sales_value',
                    aggfunc='mean',
                    fill_value=0
                    )

df = pd.pivot_table(targets_lvl_2,
                    index='user_id', columns='department',
                    values='sales_value',
                    aggfunc='mean',
                    fill_value=0
                    )

df = df.stack().reset_index()
df.columns = ['user_id', 'department', 'mean_sales_value_of_user_in_department']

targets_lvl_2 = targets_lvl_2.merge(df,
                                    on=['user_id', 'department'],
                                    how='inner')

df = \
    targets_lvl_2.groupby(by=['user_id'])['age_desc']\
    .apply(lambda x: pd.Series.mode(x))
df = df.reset_index()
df.drop(columns='level_1',
        inplace=True)

df.columns=['user_id', 'age_desc_corrected']

targets_lvl_2 = targets_lvl_2.merge(df,
                                    on='user_id',
                                    how='inner')

In [ ]:
feature_columns = \
    ['user_id', 
     'item_id', 
     'quantity', 
     'sales_value', 
     'store_id',
     'department',
     'manufacturer',
     'age_desc_corrected', 
     'brand',
     'mode_store_user',
     'quantatity_of_item_per_week',
     'quantatity_of_item_in_category_per_week', 
     'top_department',
     'top_brand', 
     'mean_sales_value_of_user_in_department'
    ]

targets_lvl_2['store_id'].fillna(mode_store, inplace=True)
targets_lvl_2[feature_columns].info()

In [ ]:
X_train = targets_lvl_2[feature_columns]
y_train = targets_lvl_2['target']
X_train[['store_id', 'mode_store_user']] = \
    X_train[['store_id', 'mode_store_user']].astype(np.int32)

In [ ]:
cat_feats = ['user_id', 'item_id', 
             'store_id', 'manufacturer', 'age_desc_corrected', 'department', 
             'brand', 'mode_store_user',
             'top_department', 'top_brand']

X_train[cat_feats] = X_train[cat_feats].astype('category')

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    random_seed=55,
    iterations=100,
    learning_rate=0.1)

model.fit(
    X_train, y_train,
    cat_features=cat_feats,
    verbose=50
)

In [ ]:
train_preds = model.predict(X_train)
train_preds = train_preds.astype(bool)

rec_items = X_train[train_preds].groupby(by=['user_id'])['item_id'].unique().reset_index()
rec_items.columns = ['user_id', 'model_preds']

rec_items['model_preds'] = \
    rec_items['model_preds'].apply(lambda x: x[:10] if len(x) >= 10 else x)

result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns = ['user_id', 'actual']
result_lvl_2.head(5)

In [ ]:
result_lvl_2 = result_lvl_2.merge(rec_items,
                                  on='user_id',
                                  how='inner')

result_lvl_2.apply(lambda row: precision_at_k(row['model_preds'], row['actual']), axis=1).mean()

### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 